In [ ]:
# 环境安装
# 1.安装并激活python环境, 已经验证的版本是3.7.2
!python --version 
!conda create -m devign python==3.7.2 #此命令不要重复执行，成功后可注释掉
!conda activate devign    
# 2.根据当前机器的情况安装torch、torch-geometric、torch-sparse 和 torch-scatter
#   具体安装方法参考 https://pytorch.org/get-started/locally/
# 3.安装当前试验所依赖的python包
!pip instlall -r requirements.txt

In [2]:
# 原始的代码数据集存放在 data/raw (Paths.raw)中
# 每一条数据包含：
# project名，commit_id，target（是否为漏洞代码），func（函数代码文本）
# 我们探索一下原始数据的构成
import pandas as pd
dataset = pd.read_json("data/raw/dataset.json")
dataset.head(5)

,project,commit_id,target,func
0,FFmpeg,973b1a6b9070e2bf17d17568cbaf4043ce931f51,0,static av_cold int vdadec_init(AVCodecContext ...
1,FFmpeg,321b2a9ded0468670b7678b7c098886930ae16b2,0,static int transcode(AVFormatContext **output_...
2,FFmpeg,5d5de3eba4c7890c2e8077f5b4ae569671d11cf8,0,"static void v4l2_free_buffer(void *opaque, uin..."
3,FFmpeg,32bf6550cb9cc9f487a6722fe2bfc272a93c1065,0,"int ff_get_wav_header(AVFormatContext *s, AVIO..."
4,FFmpeg,57d77b3963ce1023eaf5ada8cba58b9379405cc8,0,"int av_opencl_buffer_write(cl_mem dst_cl_buf, ..."


In [3]:
# 并使用joern工具将代码数据转化为CPG图
# 生成过程比较慢而且需要安装环境，在此省略
# 感兴趣的同学可以了解一下joern： https://docs.joern.io/home
# 生成的代码图数据存放在data/cpg (Paths.cpg)中，其中每100条数据写入一个文件
# 代码图数据预览
cpg_data_0 = pd.read_json("data/cpg/0_cpg.json")
print(cpg_data_0.head())
print("cpg 0:")
for k in cpg_data_0.iloc[0].functions:
    print(k, end=", ")
print("\n")
#print(cpg_data_0.iloc[0].functions["AST"])
#TODO 画出这个CPG图

                                           functions
0  {'function': 'libopus_write_header', 'file': '...
1  {'function': '<operator>.modulo', 'file': 'N/A...
2  {'function': 'av_frame_free', 'file': 'N/A', '...
3  {'function': 'th_encode_free', 'file': 'N/A', ...
4  {'function': 'put_buffer', 'file': 'N/A', 'id'...
cpg 0:
function, file, id, AST, CFG, PDG, 



In [ ]:
# 训练用于编码节点中文本的word2vec模型，并对节点的文本进行编码
from main import main
main.embed_task()

In [ ]:
# 模型训练和验证
main.process_task()

## 挑战1：完成环境配置，可以进行模型的训练和验证
## 挑战2：调整config.json中的配置，使模型的ACC达到70%
## 挑战3：原试验中代码图只使用了AST边的信息，尝试加入CFG、DFG、PDG等图的边信息，提高模型的准确率。
## 挑战4：实现一个Readout层，替代原来的Conv层
## 挑战5：embedding